# S3. Fine-tuning de modelo pre-entrenado para la clasificación de CIFAR-10
En esta sesión veremos como realizar el fine-tuning (ajuste fino) de una arquitectura de red [Inception](https://paperswithcode.com/paper/rethinking-the-inception-architecture-for), más concretamente veremos su [versión 3 (V3) disponible en Keras](https://keras.io/api/applications/inceptionv3/) pre-entrenada con la base de datos de imágenes [ImageNet](https://www.image-net.org) para la clasificación de CIFAR-10.

### Carga de datos
La utilización de una red pre-entrenada conlleva preprocesar nuestros datos con el mismo preproceso que se empleó para los datos de entrenamiento de la red pre-entrenada. En el caso de la red Inception V3, las imágenes necesitan, entre otras cosas, ser redimensionadas a 299 x 299. Si realizamos este preproceso para nuestro conjunto de entrenamiento, esto requeriría aproximadamente 40GB, por lo que es conveniente realizar este preproceso bajo demanda. Es decir, aplicaremos el preproceso para cada batch en el momento que vaya a ser utilizado.      

La aplicación de este preproceso bajo demanda está ya implementado en el módulo [tensorflow_datasets](https://www.tensorflow.org/datasets), así que en esta sesión cargaremos y manipularemos CIFAR-10 como un objeto [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) utilizando este módulo.

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
train_data, test_data = tfds.load('cifar10', split=['train', 'test'], as_supervised=True)
train_size = len(train_data)

### Preproceso

Primero definimos una función que aplica el preproceso necesario a una muestra de entrenamiento (imagen, etiqueta de clase). Esto incluye redimensionar la imagen a 299 x 299, el preproceso específico de la red InceptionV3 y convertir la etiqueta de clase a one-hot encoding.

In [3]:
from keras.applications.inception_v3 import preprocess_input

img_size = (299, 299)
num_classes = 10

def preprocess(image, label):
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)
    label = tf.one_hot(label, num_classes)
    return image, label

A continuación indicamos que la función anteriormente definida se aplicará a cada imagen cuando sea necesario. Para ello utilizamos la función [map()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) del objeto [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) y le pasamos como parámetro la función de preproceso que queremos que sea aplique a cada muestra del conjunto de datos.

In [4]:
train_data = train_data.map(preprocess)
test_data = test_data.map(preprocess)

Las funciones [take()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take) y [skip()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#skip) combinadas permiten definir los conjuntos de entrenamiento y validación como nuevos Datasets.

In [5]:
train_size = int(0.8 * train_size)
train_dataset = train_data.take(train_size)
val_dataset = train_data.skip(train_size)
test_dataset = test_data

print(len(train_dataset),len(val_dataset))

40000 10000


### Carga del modelo pre-entrenado

Seguidamente procedemos con la carga del modelo Inception V3 con los pesos resultantes de entrenarlo con la base de datos Imagenet, pero no queremos que el modelo incluya la capa de salida (include_top=False) que por defecto es una softmax de 1000 clases.

In [6]:
from keras.applications.inception_v3 import InceptionV3

model = InceptionV3(input_shape=img_size + (3,),include_top=False, weights='imagenet')
model.summary()

87910968/87910968 [==============================] - 1s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activatio

### Preparación del modelo pre-entrenado

Vamos a preparar la red Inception V3 para ser entrenada (fine-tuning) con CIFAR-10. Dado el número de parámetros de este modelo (21M), nos limitaremos a utilizarlo con los valores por defecto y añadiremos una capa GlobalAveragePooling + MLP seguida de una softmax de 10 neuronas (10 clases) acorde a CIFAR-10 que sí que entrenaremos.

In [7]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model

for layer in model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=model.input, outputs=output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalization[0][0

Compilamos el modelo con los mismos parámetros que en sesiones anteriores.

In [8]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

Entrenamos el modelo utilizando los conjuntos de datos organizado en batches y que son cargados en memoria dinámicamente.

In [9]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=10
batch_size=32
train_dataset_batched = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset_batched = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])

Epoch 1/10
1250/1250 [==============================] - ETA: 0s - loss: 0.5942 - accuracy: 0.8027
Epoch 1: val_accuracy improved from -inf to 0.85470, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1250/1250 [==============================] - 256s 192ms/step - loss: 0.5942 - accuracy: 0.8027 - val_loss: 0.4321 - val_accuracy: 0.8547 - lr: 0.0010
Epoch 2/10
1250/1250 [==============================] - ETA: 0s - loss: 0.4728 - accuracy: 0.8396
Epoch 2: val_accuracy improved from 0.85470 to 0.86940, saving model to best_model.h5
1250/1250 [==============================] - 246s 197ms/step - loss: 0.4728 - accuracy: 0.8396 - val_loss: 0.3903 - val_accuracy: 0.8694 - lr: 0.0010
Epoch 3/10
1250/1250 [==============================] - ETA: 0s - loss: 0.4408 - accuracy: 0.8483
Epoch 3: val_accuracy did not improve from 0.86940
1250/1250 [==============================] - 240s 192ms/step - loss: 0.4408 - accuracy: 0.8483 - val_loss: 0.3848 - val_accuracy: 0.8677 - lr: 0.0010
Epoch 4/10
1250/1250 [==============================] - ETA: 0s - loss: 0.4186 - accuracy: 0.8563
Epoch 4: val_accuracy improved from 0.86940 to 0.87510, saving model to best_model.h5
1250/1250 [======================

### Cargar el mejor modelo y evaluarlo con el test set

In [10]:
model = load_model('best_model.h5')
test_dataset_batched = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Test loss: 35.76
Test accuracy: 88.30


### Aumento de datos

En las sesiones anteriores, hemos utilizado la función [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) para realizar el aumento de datos. Sin embargo, esta función no se recomienda para los nuevos desarrollos de código por estar obsoleta, y en su lugar se deben utilizar las [capas de preproceso](https://www.tensorflow.org/guide/keras/preprocessing_layers). Más concretamente, utilizaremos algunas de las [capas de preproceso de aumento de datos para imágenes](https://www.tensorflow.org/guide/keras/preprocessing_layers#image_data_augmentation).

Añadiremos estas capas de aumento de datos antes del modelo Inception V3.

In [11]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, RandomRotation, RandomTranslation, RandomZoom
from keras.models import Model

input_layer = Input(shape=img_size + (3,))

x = RandomRotation(factor=0.1, fill_mode='nearest')(input_layer)
x = RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode='nearest')(x)
x = RandomZoom(height_factor=0.2, fill_mode='nearest')(x)

inception_model = InceptionV3(input_shape=img_size + (3,),include_top=False, weights='imagenet')

for layer in inception_model.layers:
    layer.trainable = False

x = inception_model(x)

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

aug_model = Model(inputs=input_layer, outputs=output)

aug_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 random_rotation (RandomRot  (None, 299, 299, 3)       0         
 ation)                                                          
                                                                 
 random_translation (Random  (None, 299, 299, 3)       0         
 Translation)                                                    
                                                                 
 random_zoom (RandomZoom)    (None, 299, 299, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0   

In [12]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
aug_model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

In [13]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=10
batch_size=32
train_dataset_batched = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset_batched = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = aug_model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])

Epoch 1/10
1250/1250 [==============================] - ETA: 0s - loss: 0.9533 - accuracy: 0.6722
Epoch 1: val_accuracy improved from -inf to 0.83980, saving model to best_model.h5
1250/1250 [==============================] - 249s 194ms/step - loss: 0.9533 - accuracy: 0.6722 - val_loss: 0.4786 - val_accuracy: 0.8398 - lr: 0.0010
Epoch 2/10
1250/1250 [==============================] - ETA: 0s - loss: 0.8102 - accuracy: 0.7197
Epoch 2: val_accuracy improved from 0.83980 to 0.85180, saving model to best_model.h5
1250/1250 [==============================] - 252s 202ms/step - loss: 0.8102 - accuracy: 0.7197 - val_loss: 0.4447 - val_accuracy: 0.8518 - lr: 0.0010
Epoch 3/10
1250/1250 [==============================] - ETA: 0s - loss: 0.7753 - accuracy: 0.7315
Epoch 3: val_accuracy improved from 0.85180 to 0.85910, saving model to best_model.h5
1250/1250 [==============================] - 243s 195ms/step - loss: 0.7753 - accuracy: 0.7315 - val_loss: 0.4242 - val_accuracy: 0.8591 - lr: 0.0010
E

In [14]:
aug_model = load_model('best_model.h5')
test_dataset_batched = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = aug_model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Test loss: 37.79
Test accuracy: 87.06


## Ejercicio:

Como habrás podido comprobar, el fine-tuning de modelos pre-entrenados a una tarea en concreto abre un amplio abanico de posibilidades para mejorar la tasa de acierto en esa tarea. Prueba a realizar fine-tuning de [otros modelos pre-entrenados para la clasificación de imágenes](https://keras.io/api/applications/) y supera la tasa de acierto del 90%.

In [2]:
from keras.applications.efficientnet_v2 import preprocess_input

img_size = (299, 299)
num_classes = 10

def preprocess(image, label):
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)
    label = tf.one_hot(label, num_classes)
    return image, label

In [3]:
train_data = train_data.map(preprocess)
test_data = test_data.map(preprocess)

In [4]:
train_size = int(0.8 * train_size)
train_dataset = train_data.take(train_size)
val_dataset = train_data.skip(train_size)
test_dataset = test_data

print(len(train_dataset),len(val_dataset))

40000 10000


In [6]:
from keras.applications.efficientnet_v2 import EfficientNetV2L

model = EfficientNetV2L(input_shape=img_size + (3,),include_top=False, weights='imagenet')
#model.summary()

In [9]:
from keras.applications.efficientnet_v2 import EfficientNetV2L
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, RandomRotation, RandomTranslation, RandomZoom
from keras.models import Model

input_layer = Input(shape=img_size + (3,))

x = RandomRotation(factor=0.1, fill_mode='nearest')(input_layer)
x = RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode='nearest')(x)
x = RandomZoom(height_factor=0.2, fill_mode='nearest')(x)

efficientNetV2L_model = EfficientNetV2L(input_shape=img_size + (3,),include_top=False, weights='imagenet')

for layer in efficientNetV2L_model.layers:
    layer.trainable = False

x = efficientNetV2L_model(x)

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

aug_model = Model(inputs=input_layer, outputs=output)

# aug_model.summary()

In [10]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
aug_model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=4
batch_size=32
train_dataset_batched = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset_batched = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = aug_model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])

Epoch 1/4


In [19]:
aug_model = load_model('best_model.h5')
test_dataset_batched = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = aug_model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Test loss: 43.06
Test accuracy: 85.43
